# Explore GeMAPS
Use this code to find out how the features of the GeMAPS feature set are extracted

Most of the logic for this Notebook is hidden in the class "Custom_GeMAPS"

In [ ]:
# Imports
# Python packages
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd

# Custom package
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent / "src"))
from age_by_voice.audio.custom_gemaps import Custom_GeMAPS, GeMAPS_Settings

# Load an audiofile
Load an audiofile into the Custom_GeMAPS class.
Its best to use a Wave file.
You can customize the samplerate.

When the file is found the class has the sample_rate (sr) and the array of samples (y).

In [ ]:
audio_path: str = "/home/chr1s/Dokumente/github/Programming/Python/Age_by_Voice/data/awed5130.wav"

# play the audio
ipd.Audio(audio_path, autoplay=True)

In [ ]:
custom_ge = Custom_GeMAPS(audio_path=audio_path)

custom_ge.sr

In [ ]:
# Plot the samples
plt.figure(figsize=(12, 4))
plt.plot(custom_ge.y)

plt.title("Audio Signal")
plt.xlabel("Sample Number")
plt.ylabel("Amplitude")

In [ ]:
# Plot a short timeframe of the audio signal 
plt.figure(figsize=(12, 4))
length = 0.02 # lenghth of the timeframe in seconds
offset = 0.5 # offset in seconds
time_start = int(custom_ge.sr * offset)
time_end = time_start + int(custom_ge.sr * length)
plt.plot(custom_ge.y[time_start:time_end])

plt.title("Audio Signal")
plt.xlabel("Sample Number")
plt.ylabel("Amplitude")

# OpenSMILE Features

Now lets inspect the features we get using the OpenSMILE library.
We use the GeMAPS_V2 Feature set and get a Pandas Dataframe from the processed audio signal

In [ ]:
smile = custom_ge.smile()
smile = smile.T # turn 90 degrees

In [ ]:
# Print the dataframe
print(smile.to_string())

# Into the Frequency domain!
Visualize the audio file in the frequency domain using the spectrogram function of the Custom_GeMAPS class.

In [ ]:
# The DFT
offset = 0.2 # offset in seconds

dft = custom_ge.dft(ts=offset, log_values=True)


plt.figure(figsize=(12, 4))
plt.plot(dft[:, 0], dft[:, 1])
plt.title("DFT of Audio Signal")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude (dB)")

In [ ]:
# The Spectrogram

spectrogram = custom_ge.spectrogram(log_values=True)

plt.figure(figsize=(12, 4))
plt.imshow(
    spectrogram.T,
    aspect="auto",
    origin="lower",
    extent=[0, custom_ge.y.shape[0] / custom_ge.sr, 0, custom_ge.sr / 2],
)
plt.title("Spectrogram")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")

In [ ]:
# Get F0
f0_dict = custom_ge.f0()
f0_mean = f0_dict["f0_mean"]
f0_stddev = f0_dict["f0_stddev"]

# print the f0 stats
# print(f"Mean F0: {f0_mean:.2f} Hz, STD: {f0_stddev:.2f} Hz")
print(f0_dict["pitch"])

In [ ]:
# Jitter
jitter_dict = custom_ge.jitter()
jitter_local = jitter_dict["jitter_local"]
jitter_local_absolute = jitter_dict["jitter_local_absolute"]
jitter_rap = jitter_dict["jitter_rap"]

# print the jitter stats
print(f"Jitter Local: {jitter_local:.5f}, Jitter Local Absolute: {jitter_local_absolute:.5f}, Jitter RAP: {jitter_rap:.5f}")

In [ ]:
# Shimmer
shimmer_dict = custom_ge.shimmer()

print(f"Shimmer Local DB: {shimmer_dict['shimmer_local_dB']:.5}")